#Library 

In [10]:
import pygad
import random
import numpy as np
import string

print(pygad.__version__)
OPTIMAL = 'GATGTGCGACTACGCAGCGGTAGCTGTGCTGCTATGGGGGGGGGGGGGGGGGGGGGGGGGTTTTTTTTTTTTTTTTTTTTTTTTTTAAAAAAAAAAAAAAAGGGGGGGTAATATATAGCAGCTGCATGCAAAAAAATTTTTTCCCCCCCCCCCCCTTCACA'

2.18.1


<Population> number of chromosome inside a population
<size_chromosome> number of genes inside a chromosome
<CreatePopulation> the funcion takes 4 letter that correspond to the oligos of the DNA, and create a Popularion of chromosome correspond to those letters

In [11]:

def createPopulation(size_chromosome, size_population):
    letters = ['A', 'G', 'T', 'C']
    population = []
    for i in range (size_population):
        direction = ''.join(random.choice(letters) for i in range(size_chromosome))
        population.append(direction)
    return population
    
population = createPopulation(5,4)
print(population)

    

['TTACC', 'CTGTA', 'GGTTC', 'TAGAG']


<fiteness> we pass a chromosome and we check with this simple function which is the distance between the selected cromosome and the one that we want to generate
<ord>it return the ASCII value associated with the letter of the chromosome passed as imput, it is the same for the optimal


In [12]:
def fitness(chromosome):
    """
    For each gene in the DNA, this function calculates the difference between
    it and the character in the same position in the OPTIMAL string
    These values are summed and then returned.
    The closer the DNA is to the OPTIMAL, the lower the fitness.
    """
    fit = 0
    for i in range(len(chromosome)):
        fit += abs(ord(chromosome[i]) - ord(OPTIMAL[i]))
    return fit

fit = fitness("TTTTA")
print(fitness)

<function fitness at 0x0000027BD6AF9310>


In [13]:
def random_char(U):
    """
    Return a random character A, C, G, T, U based on ASCII values. 85
    """
    if U == 0:
        mutation = chr(random.choice([65, 67, 71, 84]))
    else:
        mutation = chr(random.choice([65, 67, 71, 84, 85]))
    return mutation
    
name = []
for i in range(3):
    name.append(random_char(1)) 
print(name)



['C', 'T', 'A']


In [14]:
def mutate(dna, probabilty_mutation):
    """
    For each gene in the DNA, there is a 1/mutation_chance chance that it will be
    switched out with a random character. This ensures diversity in the
    population, and ensures that it is difficult to get stuck in local minima.
    """
    dna_out = ""
    
    for c in range(len(dna)):
        if round(random.random() * probabilty_mutation) == 1: # 0.5 * 5 = 
            dna_out += random_char(1) # generate a random character to swap
        else:
            dna_out += dna[c]
    return dna_out



In [15]:
def weighted_choice(items):
    
    weight_list = []
    weight_total = sum((item[1] for item in items))
    
    for i, w in items:
        weight_list.append(w / weight_total)
    
    n = random.choices(items, weights = weight_list, k = 1)

    return n[0][0]



In [16]:
def crossover(dna1, dna2):
    """
    Slices both dna1 and dna2 into three parts at a random index within their
    length and merges them. Both keep their initial sublist up to the crossover
    index, but their ends are swapped.
    """
    parent1_or_parent2 = random.randint(1,2) #choose one of the 2 parent
    
    full_piece = []
    
    for i in range(0,2): #take 2 number and order them
        full_piece.append(random.randint(1, len(dna1)))

    minimum = min(full_piece)
    maximum = max(full_piece)

    if (parent1_or_parent2 == 2):
        parent1 = dna1[0:minimum] + dna2[minimum:maximum] + dna1[maximum:]
        parent2 = dna2[0:minimum] + dna1[minimum:maximum] + dna2[maximum:]
    else:
        parent1 = dna2[0:minimum] + dna1[minimum:maximum] + dna2[maximum:]
        parent2 = dna1[0:minimum] + dna2[minimum:maximum] + dna1[maximum:]

    return(parent1, parent2)

var = crossover("abcdef", "ghilmn")

In [17]:
def generate_population(generation, optimal, lenght_chromosome, number_chromosome):
    
    population = createPopulation(lenght_chromosome,number_chromosome)
    weighted_population = []
    correct_generation = 0

    for G in range(0 , generation):
        weighted_population = []
        sum_fitness = 0
        
        for I in population:
            
            fit_value = fitness(I)

            if I == optimal:
                exact_population = population
                correct_generation = G
                return(exact_population, correct_generation)
            
            if fit_value != 0:
                pair = (I, 1 / fit_value)
                sum_fitness = sum_fitness + ( 1 / fit_value)
            else:
                pair = (I, 1)

            weighted_population.append(pair)
            
        print(f'Generation number --> {G} Fitness sum --> {sum_fitness} ')
        population = []
        prob_of_mutation = 0
            # Select two random individuals, based on their fitness probabilities,
            # cross their genes over at a random point, mutate them, and add them
            # back to the population for the next iteration.
            
        for _ in range(round(len(weighted_population) / 2)):
            
            if (G % 100 == 0):
                prob_of_mutation += 1
            # Selection
            ind1 = weighted_choice(weighted_population)
            ind2 = weighted_choice(weighted_population)
            
            # Crossover
            ind1, ind2 = crossover(ind1, ind2)

            # Mutate and add back into the population
            population.append(mutate(ind1,prob_of_mutation))
            population.append(mutate(ind2,prob_of_mutation))
    
    print(f'the correct generation is: {correct_generation}')
    return(population, correct_generation)

pop, gen = generate_population(10000, OPTIMAL, len(OPTIMAL), 10000)

print(pop,gen)
            
    




Generation number --> 0 Fitness sum --> 7.844583212908283 
Generation number --> 1 Fitness sum --> 7.87730805687339 
Generation number --> 2 Fitness sum --> 7.919758760790626 
Generation number --> 3 Fitness sum --> 7.954754923344373 
Generation number --> 4 Fitness sum --> 7.987931459230134 
Generation number --> 5 Fitness sum --> 8.024115413551613 
Generation number --> 6 Fitness sum --> 8.07001391011618 
Generation number --> 7 Fitness sum --> 8.112194651744945 
Generation number --> 8 Fitness sum --> 8.15524068606208 
Generation number --> 9 Fitness sum --> 8.192108134983279 
Generation number --> 10 Fitness sum --> 8.233145287883675 
Generation number --> 11 Fitness sum --> 8.268461003302612 
Generation number --> 12 Fitness sum --> 8.307428136910596 
Generation number --> 13 Fitness sum --> 8.358906572611641 
Generation number --> 14 Fitness sum --> 8.409981361528915 
Generation number --> 15 Fitness sum --> 8.447272079685378 
Generation number --> 16 Fitness sum --> 8.4857468271